In [4]:
# Sample code to perform I/O:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import os
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
master_train=train
master_test=test
#print(train_data.head())
# Warning: Printing unwanted or ill-formatted data to output will cause the test cases to fail

# Write your code here
## Exploratory Data Analysis
### We will first check the distribution of data of the Target Variable. i.e. Problem Category

print('Target Variable Distribution - ',train['Problem Category'].value_counts())
plt.figure(figsize=(10,10))
plt.xticks(rotation=90)
plt.title('Distribution of Target Variable')
sns.countplot(x='Problem Category',data=train)

# Inference: As understood from the graph - People are asking a lot about Problem Category 1 i.e. details regarding their own house.Questions related to Problem Category 2 and 0 are not that much 

## Pairplot of seaborn package gives a comprehensive evaluation on what all are there and what is the trend between variables.
## Pairplot works wonders and easier to compute when the no. of columns are less. In our condition,our columns would be ok to compute
## and won't take much time. We have removed the following variables. -- Date related variables as I think it wont help much for now. 
## If I have some time, I was thinking of using feature selection to compute the time taken the task took to complete from the time
## taken the task was raised. But for now, we are using the variables.

variable_list=['Agent Category Assigned','Type of Request','Description of the Request','Location','Region Type','Ward No','Request Solution Category','Team Assigned','A_1','A_2']
label=master_train['Problem Category']
Id_test=master_test['L_Id']
train=master_train[variable_list]
test=master_test[variable_list]

## sns.pairplot(train)
## The above plot takes a lot to compute - However, I had run the above locally in my system. There was not much trends that were
## showing up . One thing that was interesting is that the variables A_1 and A_2 are closely related. To prove the relation, we can 
## create a heatmap to showcase variables that are related.

corr_data=train.corr()
sns.heatmap(corr_data,annot=True)

## As we can see from the heatmap, the variables A_1 and A_2 are highly correlated, therefore we need to remove it 
## By firing the below command we also find that there is only one value for Agent Category Assigned, therefore, we are 
## going to remove that as well

#for variable_name in variable_list:
    #print(variable_name,' - ',train[variable_name].value_counts())

variable_list=['Type of Request','Description of the Request','Location','Region Type','Ward No','Request Solution Category','Team Assigned','A_1']

train=master_train[variable_list]
test=master_test[variable_list]

## Since the count of the problem Category 1 is the highest, we will now dive into the data distribution and check if they have any 
## patterns.However, I checked some of the variables and their distribution, I was not able to identify such patterns.

ax_1=sns.scatterplot(x='Ward No',y='Team Assigned',data=master_train[master_train['Problem Category']==1])

ax_2=sns.scatterplot(x='Description of the Request',y='Type of Request',data=master_train[master_train['Problem Category']==1])

## Please note: The below code was working fine in my local machine but for some reason,it was not working in here 
## so , I have commented it. The functionality of below code is to find the null values and then replace it using the fill na 
## function

#list of columns with null values -train
#list_of_clm_null_value=train.columns[train.isna().any()].tolist()
#print(list_of_clm_null_value)

null_columns=train.columns[train.isnull().any()]
count=train[null_columns].isnull().sum()
#print(count)

#for i in list_of_clm_null_value:
    #train[i].fillna(-1,inplace=True)

#list of columns with null values -test
#list_of_clm_null_value=test.columns[test.isna().any()].tolist()
#print(list_of_clm_null_value)

null_columns=test.columns[test.isnull().any()]
count=test[null_columns].isnull().sum()
#print(count)

#for i in list_of_clm_null_value:
    #test[i].fillna(-1,inplace=True)

## This dataset is clear cut case of data imbalance. Therefore, since the weightage becomes high of the majority.
## We are using SMOTE to even the playing field and build a better model
## This was one technique. 

## We tested other models as well out of which decision tree was giving a comparative good score as compared to 
## NaiveBayes,RandomForest`,Logistic Regression etc. 
## We had ran the models in our local systems
## SMOTE stand for Synthetic Minority Oversampling Technique
## It is a very popular oversampling method that was proposed to improve random oversampling .
## If we choose to undersample, there may be a chance of losing critical data that helps in defining the relationship between the 
## target variable and predictor variables for which we are choosing over-sampling. 

smote=SMOTE(sampling_strategy='all',random_state=47)
os_values,os_labels=smote.fit_sample(train,label)

features_train=pd.DataFrame(os_values)
labels_train=pd.DataFrame(os_labels)

DT=DecisionTreeClassifier()
y_pred_DT=DT.fit(features_train,labels_train.values.ravel()).predict(test)



clf=DecisionTreeClassifier()
param_grid={
        'criterion':['entropy','gini'],
        'max_depth':[4,7,10,15,20,25,40],
        'min_samples_split':[2,6,8,10],
        'random_state':[47]
}

clf=GridSearchCV(clf,param_grid)

y_pred_DT_tune=clf.fit(features_train,labels_train.values.ravel()).predict(test)

##I am also trying XGBoost to check if the model becomes good or not. 

xgb=XGBClassifier()
y_pred_xgb=xgb.fit(train,label).predict(test)

##XGBoost was showing similar results but since we had tuned the DT, we will use that as our final submission

df=pd.DataFrame()
df['L_Id']=Id_test
df['Problem Category']=y_pred_DT_tune


#If 'df' is the dataframe containing your predictions and no feature column is for index:
df.to_csv('submission.csv', index=False)

Here
['Street Type']
Street Type    419
dtype: int64
['Street Type']
Street Type    172
dtype: int64


1    2498
0     125
2      53
Name: Problem Category, dtype: int64